## Food Delivery Time Prediction Model

You will train a ML model using an LSTM neural network model for the food delivery time prediction. It constructs a neural network using Keras for a food delivery system, designed to predict delivery times based on inputs like age, rating, and distance. :

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from kafka import KafkaConsumer
import plotly.express as px
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio

model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (3, 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error', run_eagerly=True)
model.summary()

It starts with a Sequential model, indicating a linear stack of layers, followed by two LSTM (Long Short-Term Memory) layers - ideal for sequential data processing. 

Followed by two Dense layers: the first with 25 neurons for further processing, and the second with a single neuron to output the predicted delivery time. 

The model uses the 'adam' optimizer and 'mean_squared_error' as the loss function, suitable for regression tasks like predicting time. 

Finally, model.summary() provides a structured overview of this neural network architecture.

## Continuous Data Training in Machine Learning

You will now start the training by running the next snippet, 

In [ ]:
online_train_ds = tfio.experimental.streaming.KafkaBatchIODataset(
    topics=["model-data"],
    group_id="testzo",
    servers="redpanda-0:9092,redpanda-1:9092,redpanda-2:9092",
    stream_timeout=10000,
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=earliest"
    ],
)


def decode_kafka_online_item(raw_message, raw_key):
    message = tf.io.decode_csv(raw_message, [[0.0] for i in range(3)])
    key = tf.strings.to_number(raw_key)
    return (message, key)
  
batch_size = 20
for single_ds in online_train_ds:
    if len(single_ds) >= batch_size:
        single_ds = single_ds.shuffle(buffer_size=batch_size)
        single_ds = single_ds.map(decode_kafka_online_item)
        single_ds = single_ds.batch(batch_size)
    
        model.fit(single_ds, epochs=1)
        tf.keras.models.save_model(model, "./time_prediction_model")
    else:
        print("Not enough data in the dataset. Skipping model fitting.")




The above machine learning code leverages **TensorFlow** I/O’s __KafkaBatchIODataset__ to stream data directly from Redpanda topics into a TensorFlow dataset. This dataset is configured to ingest data from the 'model-data' topic on a Redpanda cluster. The main processing loop handles data in batches: it accumulates messages until there are enough to constitute a batch (set to a size of 20), then shuffles and decode batch before using it for training. Subsequently, the model is trained for one epoch with each batch and then saved.

This ongoing training loop enables the model to dynamically learn from a continuous stream of real-time food delivery data. Such an approach ensures that applications utilizing this model can provide predictions that are consistently updated with the most recent data.